## H2O Model Trainer
(max rows limit/threshold - leader model metrics)

In this notebook, once data is entered, the max rows threshold (limit_rows) is applied to the dataset to avoid overfitting or very large datasets. Then, H2OAutoML is trained with only these 3 algos: GBM, GLM and XGBoost. Once training is done, the leader model's metrics are extracted, if they are less than the threshold fixed, another H2OAutoML model is trained using DeepLearning algo. The best out of the two leaders is saved as the model to use.

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 913.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 1.9 MB/s e

In [2]:
!ngrok authtoken '2ighL0YEwJxisFZFo8JWIFL1wtf_3CdEFhapKNHeoHFAE2m4d'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
from io import StringIO
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import logging
import os

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.23" 2024-04-16; OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprnalzqz4
  JVM stdout: /tmp/tmprnalzqz4/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprnalzqz4/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_unknownUser_3qy1u8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [5]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [6]:
@app.get("/")
async def root():
    return "Hello World!"

In [7]:
def limit_rows(df, max_rows=500):
    num_rows, num_cols = df.shape
    df_non_null = df.dropna()

    if len(df_non_null) > max_rows:
        df_non_null = df_non_null.sample(n=max_rows, random_state=42)
        print("num rows extracted (non-null): ", max_rows)
    else:
        print("num rows: ",len(df_non_null))

    print(df_non_null)
    return df_non_null

In [8]:
@app.post('/train')
async def train_model(file: UploadFile = File(...)):
    csv_data = StringIO((await file.read()).decode('utf-8'))
    df = pd.read_csv(csv_data)
    df = limit_rows(df)
    h2o_df = h2o.H2OFrame(df)
    x = h2o_df.columns
    y = x[-1]
    x.remove(y)

    prob_type = "regression"

    target_unique_values = h2o_df[y].unique().nrow
    if (h2o_df[y].isnumeric()[0] and target_unique_values < 10) or not h2o_df[y].isnumeric()[0]:
        prob_type = "classification"
        h2o_df[y] = h2o_df[y].asfactor()

    include_algos = ["GLM", "GBM", "XGBoost"]

    aml = H2OAutoML(max_models=10, seed=1, include_algos=include_algos)
    aml.train(x=x, y=y, training_frame=h2o_df)
    print(aml.leader)

    model = aml.leader
    model_metrics = model.model_performance()._metric_json

    include_dl = False
    if prob_type == "classification" :
      if float(model_metrics['logloss'])>0.2:
        include_dl = True
    elif float(model_metrics['r2'])<0.8:
      include_dl = True

    if include_dl :
        metric = model_metrics['logloss'] if prob_type == "classification" else model_metrics['r2']
        print("DeepLearning included, metric : ",metric)
        aml2 = H2OAutoML(max_models=2, seed=1, include_algos=["DeepLearning"])
        aml2.train(x=x, y=y, training_frame=h2o_df)
        print(aml2.leader)
        model2 = aml2.leader
        if prob_type == "classification" :
          if float(model2.model_performance()._metric_json['logloss'])<float(model_metrics['logloss']):
            model = model2
        elif float(model2.model_performance()._metric_json['r2'])>float(model_metrics['r2']):
            model = model2

    model_path = h2o.save_model(model=model, path="./models", force=True)
    model_metrics = model.model_performance()._metric_json

    if prob_type == "classification" :
      model_details = {
          'model_id': model.model_id,
          'model_type': model.algo,
          'model_path': model_path,
          'model_category': model_metrics['model_category'],
          'AUC' : model_metrics['AUC'],
          'logloss' : model_metrics['logloss'],
          'MSE' : model_metrics['MSE'],
      }
    else :
      model_details = {
          'model_id': model.model_id,
          'model_type': model.algo,
          'model_path': model_path,
          'model_category': model_metrics['model_category'],
          'MSE' : model_metrics['MSE'],
          'RMSE' : model_metrics['RMSE'],
          'R2' : model_metrics['r2']
      }

    return JSONResponse(content={'modelpath': model_path, 'model_details': model_details})

In [9]:
#mae/y.mean() or r2>0.95 or mape

In [10]:
@app.post('/predict')
async def predict_model(modelpath: str = Form(...), file: UploadFile = File(...)):
    csv_data = StringIO((await file.read()).decode('utf-8'))
    input_df = pd.read_csv(csv_data)
    h2o_input_df = h2o.H2OFrame(input_df)

    model = h2o.load_model(modelpath)

    predictions = model.predict(h2o_input_df)
    predictions_df = predictions.as_data_frame()

    return JSONResponse(content=predictions_df.to_dict(orient="records"))

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [227]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://080e-34-106-25-126.ngrok-free.app
num rows extracted (non-null):  500
          id  age  gender driving_experience    education        income  \
3265  384446    2       1             20-29y   university   upper class   
603   456321    2       0               0-9y         none       poverty   
9998  903459    1       0             10-19y  high school       poverty   
9984  443302    1       0             10-19y  high school  middle class   
4695  371790    1       0             10-19y  high school  middle class   
...      ...  ...     ...                ...          ...           ...   
9465  487682    2       1             20-29y   university   upper class   
8814  783337    0       0               0-9y  high school  middle class   
493   881409    3       1               30y+   university   upper class   
8794  798069    2       1               0-9y   university   upper class   
3598  955335    0       1               0-9y         none       poverty   

      cre

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


num rows extracted (non-null):  500
      Age  Gender        BMI  Smoking  GeneticRisk  PhysicalActivity  \
1116   46       1  30.193803        0            1          7.111218   
1368   49       1  33.547408        0            2          3.047609   
422    73       0  15.604794        0            0          6.579499   
413    41       0  25.247622        0            0          6.860912   
451    60       0  22.054677        1            0          7.804711   
...   ...     ...        ...      ...          ...               ...   
591    51       1  36.454426        0            2          4.498040   
664    43       0  15.450164        1            0          7.089000   
195    42       1  15.275782        0            2          7.405095   
1240   75       1  38.785745        0            2          9.484252   
1048   21       0  21.785235        0            0          8.355717   

      AlcoholIntake  CancerHistory  Diagnosis  
1116       2.770849              1          1  
136

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


...

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


num rows extracted (non-null):  500
         x          y
158  100.0  96.623279
500   97.0  94.296334
397   12.0  14.558961
155   86.0  86.821321
322   91.0  94.367790
..     ...        ...
586   98.0  98.613203
349   35.0  34.785610
464    0.0   2.116113
326   69.0  71.256341
186   97.0  96.498124

[500 rows x 2 columns]
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_14_20240722_171426


GLM Model: summary
    family    link      regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  ---------------------------  --------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


num rows extracted (non-null):  500
         x          y
158  100.0  96.623279
500   97.0  94.296334
397   12.0  14.558961
155   86.0  86.821321
322   91.0  94.367790
..     ...        ...
586   98.0  98.613203
349   35.0  34.785610
464    0.0   2.116113
326   69.0  71.256341
186   97.0  96.498124

[500 rows x 2 columns]
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_15_20240722_171746


GLM Model: summary
    family    link      regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  --------  ---------------------------  --------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
